# Script to run LePhare on a matched table

## Requirements

This notebook is supposed to be run on a matched LePhare input file after carrying out the matching using the `match_tables.ipynb` notebook.

## Structure

- Load the region and the relevant matched and processd files
- Running `LePhare`:
  1. Collect the filter information
  2. Collect the template information
     1. For the stellar library
     2. For the pointlike library
     3. For the extended library
  3. Run the photo-z-routine

In [1]:
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")
# Ignore the 'Could not import regions' line, it is from one of the submodules

INFO:root:Modules loaded successfully


Could not import regions, which is required for some of the functionalities of this module.


In [2]:
# Load the region that has been saved before. If this fails, make sure to run `match_tables.ipynb` first!
REGION = fp.Region(load_from_disk=True)
logging.info(REGION)

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2


## Run LePhare

Before we run any of the subprograms, let us initialise important stems that we're going to use over and over:

In [3]:
LEPHAREDIR = fp.get_lephare_directory("dir")
LEPHAREWORK = fp.get_lephare_directory("work")
PARAFILE = fp.get_filepath("para_in")
PARA_OUT = fp.get_filepath("para_out")

FILTER_STEM = "ls10plus"

STAR_STEM = "base"
STAR_FILE = fp.get_filepath("template", "star", stem=STAR_STEM)
STAR_SEDLIB_NAME = f"{STAR_STEM}_star_sedlib"
STAR_MAGLIB_NAME = f"{STAR_STEM}_star_maglib"
TEMP_STEM = "combined"  # This refers to the templates that seemed to work best in my thesis
PLIKE_FILE = fp.get_filepath("template", "pointlike", stem=TEMP_STEM)
PLIKE_SEDLIB_NAME = f"{TEMP_STEM}_pointlike_sedlib"
PLIKE_MAGLIB_NAME = f"{TEMP_STEM}_pointlike_maglib"
EXT_FILE = fp.get_filepath("template", "extended", stem=TEMP_STEM)
EXT_SEDLIB_NAME = f"{TEMP_STEM}_extended_sedlib"
EXT_MAGLIB_NAME = f"{TEMP_STEM}_extended_maglib"

RUN_STEM = "base"
PLIKE_IN = fp.get_filepath("lephare_in", "pointlike", stem=RUN_STEM)
PLIKE_OUT = fp.get_filepath("lephare_out", "pointlike", stem=RUN_STEM)
EXT_IN = fp.get_filepath("lephare_in", "extended", stem=RUN_STEM)
EXT_OUT = fp.get_filepath("lephare_out", "extended", stem=RUN_STEM)


### 1) Run the filter file

> **Notes**:\
> This step can be skipped if the filterfile has already been set up before.\
> If you wanted to change the bands used, you need to change some parameters in the parameter file as well!\
> Also, make sure to rerun the commands to produce the magnitude libraries in this case!

In [4]:
# We use this stem for the Galex, sweep (+WISE) and VHS filters:
# FUV, NUV, g, r, i, z, J, H, Y, Ks, W1, W2, W3, W4
FILTER_OUT = fp.get_filepath("filter", stem=FILTER_STEM)
FILTER_REPO = fp.get_lephare_directory("filter_origin")

In [18]:
!"$LEPHAREDIR"/source/filter -c "$PARAFILE" \
    -FILTER_REP "$FILTER_REPO" \
    -FILTER_FILE "$FILTER_STEM" \
    >"$FILTER_OUT"

### 2) Run the template files

Here, we create the template files that LePhare uses to fit the SEDs against.\
The creation usually consists of two steps:
1. The creation of the SED library:\
   After we've provided a list of the filenames where our template files are located, LePhare unifies them to a single library file.\
   This is done via the `sedtolib` command.
2. The creation of the magnitude library:\
   Then, these library SEDs have to be convolved through each of our filters to create physical magnitudes we would observe.\
   These are then propagated in a redshift grid, and, if requested, an EBV-grid to create the magnitude libraries that are later used for the fits.\
   This is done via the `mag_gal` command.

> **Notes**:\
> These steps can be skipped if the required template files already exist.\
> Be sure to repeat this step if using a different set of filters!\
> If you're introducing new template lists, make sure they're included in the ``data/lephare/templates`` directory!

#### 2a) The stellar template library

In [20]:
# This is the sedtolib routine to create the general stellar library.
!"$LEPHAREDIR"/source/sedtolib -c "$PARAFILE" \
    -t S \
    -STAR_SED "$STAR_FILE" \
    -STAR_LIB "$STAR_SEDLIB_NAME"

Number of keywords read at the command line (excluding -c config): 3
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword SEL_AGE not provided 
Keyword AGE_RANGE not provided 
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Library type     : STAR
# STAR_SED    :/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/base_star.list
# STAR_LIB    :base_star_sedlib
# STAR_LIB doc:/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/base_star_sedlib.doc
# STAR_FSCALE :3.432e-09
#######################################
Number of SED in the list 254

Times to run the code in sec: 1.85839


In [21]:
# This is the mag_gal routine to create the convolved stellar magnitude library

!"$LEPHAREDIR"/source/mag_gal -c "$PARAFILE" \
    -t S \
    -STAR_LIB_IN "$STAR_SEDLIB_NAME" \
    -STAR_LIB_OUT "$STAR_MAGLIB_NAME" \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE "$FILTER_STEM"

Running with OpenMP on 48 threads!
Number of keywords read at the command line (excluding -c config): 6
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword EXTINC_LAW not provided 
Keyword EB_V not provided 
Keyword EM_DISPERSION not provided 
 All files opened 
#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Filter file     : ls10plus
# Magnitude type     : AB
# STAR_LIB_IN    :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/base_star_sedlib(.doc & .bin)
# STAR_LIB_OUT   :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_mag/base_star_maglib(.doc & .bin)
# LIB_ASCII   YES
# CREATION_DATE Tue Nov  1 21:10:20 2022
#############################################
 Read SED MOD_254 z      0  ead SED MOD

#### 2b) The pointlike library

In [22]:
# This is the sedtolib routine to create the general pointlike library.
!"$LEPHAREDIR"/source/sedtolib -c "$PARAFILE" \
    -t G \
    -GAL_SED "$PLIKE_FILE" \
    -GAL_LIB "$PLIKE_SEDLIB_NAME"

Number of keywords read at the command line (excluding -c config): 3
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword SEL_AGE not provided 
Keyword AGE_RANGE not provided 
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Library type     : GAL
# GAL_SED    :/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/combined_pointlike.list
# GAL_LIB    :combined_pointlike_sedlib
# GAL_LIB doc:/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_pointlike_sedlib.doc
# GAL_LIB phys:/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_pointlike_sedlib.phys
# SEL_AGE    :none
# GAL_FSCALE :1
# AGE_RANGE   -999 -999
#######################################
Number of SED in the list 16

Times to run the code in sec: 0.142556


In [23]:
# This is the mag_gal routine to create the convolved pointlike magnitude library
# The last three lines correspond to the prescribed extinction laws, the models they should be applied on
# and the EB_Vs to consider for each application.

!"$LEPHAREDIR"/source/mag_gal -c "$PARAFILE" \
    -t G \
    -GAL_LIB_IN "$PLIKE_SEDLIB_NAME" \
    -GAL_LIB_OUT "$PLIKE_MAGLIB_NAME" \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE "$FILTER_STEM" \
    -EXTINC_LAW SMC_prevot.dat,SB_calzetti.dat \
    -MOD_EXTINC 0,0,0,30 \
    -EB_V 0.,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4

Running with OpenMP on 48 threads!
Number of keywords read at the command line (excluding -c config): 9
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword EM_DISPERSION not provided 
 All files opened 
#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Filter file     : ls10plus
# Magnitude type     : AB
# GAL_LIB_IN    :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_pointlike_sedlib(.doc & .bin)
# GAL_LIB_OUT   :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_mag/combined_pointlike_maglib(.doc & .bin)
# ZGRID_TYPE   :0
# Z_STEP   :0.02 0 6
# COSMOLOGY   :70 0.3 0.7
# EXTINC_LAW   :SMC_prevot.dat SB_calzetti.dat 
# MOD_EXTINC   :0 0 0 30 
# EB_V   :0 0.05 0.1 0.15 0.2 0.25 0.3 0.35 0.4 


#### 2c) The extended library

In [24]:
# This is the sedtolib routine to create the general extended library.
!"$LEPHAREDIR"/source/sedtolib -c "$PARAFILE" \
    -t G \
    -GAL_SED "$EXT_FILE" \
    -GAL_LIB "$EXT_SEDLIB_NAME"

Number of keywords read at the command line (excluding -c config): 3
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword SEL_AGE not provided 
Keyword AGE_RANGE not provided 
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Library type     : GAL
# GAL_SED    :/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/combined_extended.list
# GAL_LIB    :combined_extended_sedlib
# GAL_LIB doc:/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_extended_sedlib.doc
# GAL_LIB phys:/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_extended_sedlib.phys
# SEL_AGE    :none
# GAL_FSCALE :1
# AGE_RANGE   -999 -999
#######################################
Number of SED in the list 40

Times to run the code in sec: 0.351302


In [25]:
# This is the mag_gal routine to create the convolved extended magnitude library
# We don't use extinction laws here.

!"$LEPHAREDIR"/source/mag_gal -c "$PARAFILE" \
    -t G \
    -GAL_LIB_IN "$EXT_SEDLIB_NAME" \
    -GAL_LIB_OUT "$EXT_MAGLIB_NAME" \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE "$FILTER_STEM"

Running with OpenMP on 48 threads!
Number of keywords read at the command line (excluding -c config): 6
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword EXTINC_LAW not provided 
Keyword EB_V not provided 
Keyword EM_DISPERSION not provided 
 All files opened 
#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# Filter file     : ls10plus
# Magnitude type     : AB
# GAL_LIB_IN    :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_bin/combined_extended_sedlib(.doc & .bin)
# GAL_LIB_OUT   :/hs/fs14/data/4most/4hi-q/LEPHARE/work/lib_mag/combined_extended_maglib(.doc & .bin)
# ZGRID_TYPE   :0
# Z_STEP   :0.02 0 6
# COSMOLOGY   :70 0.3 0.7
# EXTINC_LAW   :calzetti.dat 
# MOD_EXTINC   :0 0 
# EB_V   :0 
#

> **Note**:\
> If you want to analyse the template files, they can be relocated to the 'template' directory as follows:

In [27]:
for filename in [STAR_MAGLIB_NAME, PLIKE_MAGLIB_NAME, EXT_MAGLIB_NAME]:
    filepath = fp.get_lephare_directory("work") + "lib_mag"
    sourcepath = f"{filepath}/{filename}.dat"
    destpath = fp.get_lephare_directory("templates") + filename + ".dat"
    # Simply use the Linux routine to copy the files:
    if fp.ask_file_overwrite(destpath):
        os.system(f"cp {sourcepath} {destpath}")
logging.info("All files copied successfully.")

INFO:root:Writing the file '/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/base_star_maglib.dat'
INFO:root:Skipped writing '/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/combined_pointlike_maglib.dat'
INFO:root:Skipped writing '/hshome/bat3083/sel_4hi_q_v2/data/lephare/templates/combined_extended_maglib.dat'
INFO:root:All files copied successfully.


### 3) Run the photo-z routine

This is the core functionality of LePhare.\
Be sure you know what you're doing when modifying the parameters.

#### 3a) Run the photo-zs for the pointlike sources

In [28]:
# Perform the zphota routine:
!"$LEPHAREDIR"/source/zphota -c "$PARAFILE" \
    -ZPHOTLIB "$PLIKE_MAGLIB_NAME","$STAR_MAGLIB_NAME" \
    -CAT_IN "$PLIKE_IN" \
    -CAT_OUT "$PLIKE_OUT" \
    -PARA_OUT "$PARA_OUT" \
    -LIB_ASCII YES \
    -MAG_REF 7 \
    -MAG_ABS -30,-20
# Additional sometimes useful parameters:
# -PDZ_OUT YES  -> Generate .spec files for each source
# -SPEC_OUT YES  -> Generate .spec files for each source
# -CAT_LINES 10, 10  -> Only consider the first 10 rows (highly recommended with .spec files)
# -GLB_CONTEXT context  -> Limit the SED-Fitting to a subset of the bands described via 'context'


Starting times in sec: 0.494496
local:     Tue Nov  1 21:19:42 2022

Number of keywords read at the command line (excluding -c config): 7
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword MASS_SCALE not provided 
Keyword NZ_PRIOR not provided 
Keyword RM_DISCREPENT_BD not provided 
Keyword PROB_INTZ not provided 
Keyword M_REF not provided 
Keyword APPLY_SYSSHIFT not provided 
Wrong number of arguments for option MABS_ZBIN
We have 8 instead of 5
Use default value 
Wrong number of arguments for option MABS_ZBIN
We have 8 instead of 5
Use default value 
####################################### 
# PHOTOMETRIC REDSHIFT with OPTIONS   # 
# Config file            : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# CAT_IN                 : /hshome/bat3083/sel_4hi_q_v2/data/lephare/input/base_input_pointlike.in
# CAT_OUT                : /hshome/bat3083/sel_4hi_q_v2/data/lephare/outpu

#### 3b) Run the photo-zs for the extended sources

In [29]:
# Perform the zphota routine:
!"$LEPHAREDIR"/source/zphota -c "$PARAFILE" \
    -ZPHOTLIB "$EXT_MAGLIB_NAME","$STAR_MAGLIB_NAME" \
    -CAT_IN "$EXT_IN" \
    -CAT_OUT "$EXT_OUT" \
    -PARA_OUT "$PARA_OUT" \
    -GLB_CONTEXT -1 \
    -LIB_ASCII YES \
    -MAG_REF 7 \
    -MAG_ABS -24,-8
# Additional sometimes useful parameters:
# -PDZ_OUT YES  -> Generate .spec files for each source
# -SPEC_OUT YES  -> Generate .spec files for each source
# -CAT_LINES 10, 10  -> Only consider the first 10 rows (highly recommended with .spec files)
# -GLB_CONTEXT context  -> Limit the SED-Fitting to a subset of the bands described via 'context'


Starting times in sec: 0.492864
local:     Tue Nov  1 21:21:50 2022

Number of keywords read at the command line (excluding -c config): 8
Reading keywords from /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
Number of keywords read in the config file: 94
Keyword MASS_SCALE not provided 
Keyword NZ_PRIOR not provided 
Keyword RM_DISCREPENT_BD not provided 
Keyword PROB_INTZ not provided 
Keyword M_REF not provided 
Keyword APPLY_SYSSHIFT not provided 
Wrong number of arguments for option MABS_ZBIN
We have 8 instead of 5
Use default value 
Wrong number of arguments for option MABS_ZBIN
We have 8 instead of 5
Use default value 
####################################### 
# PHOTOMETRIC REDSHIFT with OPTIONS   # 
# Config file            : /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
# CAT_IN                 : /hshome/bat3083/sel_4hi_q_v2/data/lephare/input/base_input_extended.in
# CAT_OUT                : /hshome/bat3083/sel_4hi_q_v2/data/lephare/output